### Building a Tree Code IV: Trying Out the Tree Walk

In this series of notebooks we will build a tree code for determining the acceleration from a set of "particles" (point masses) in $O(N\log N)$ time.

In this part we will try it out on a Plummer model.

In [ ]:
%matplotlib inline

from dataclasses import dataclass

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
import time

import numpy.random as rng
rng.seed(24238929)

from Nbody import ParticleSet, Morton, TreeNode, Octree, BHtree

from plummer import PlummerModel

# make notebook as wide as browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
plt.rcParams['figure.dpi'] = 150

A commonly-used test problem is the Plummer sphere, and equilibrium distribtion with a density
$$ \rho(r) = \frac{3M}{4\pi a^3}\left[1 + \frac{r^2}{a^2}\right]^{-5/2} $$
so that the potential is
$$ \Phi(r) = -\frac{GM}{(r^2 + a^2)^{1/2}} $$
and the acceleration is
$$ |\mathbf{a}(r)| = -GM \frac{r}{(r^2 + a^2)^{3/2}} $$

We can Monte Carlo sample from a Plummer sphere. Each sample is a "star". Because the density reaches zero only at $r\rightarrow\infty$,
we need to impose a cut-off in radius. The acceptance-rejection method commonly used was first describd in an appendix to Aarseth, S. J., Henon, M., & Wielen, R. *A Comparison of Numerical Methods for the Study of Star Cluster Dynamics*,
A&A 37, p.183 (1974).

Let's try it out on a Plummer sphere of 16K points. I haven't quite figured out the normalization for
the Python Plummer sphere method I grabbed off github... I'll compare it with my C++ version soon...

In [ ]:
N = 2**14
p = ParticleSet(N)
p.pos[:,:], p.vel[:,:], p.mass[:] = PlummerModel(N)
p.pos /= 4*np.pi**2
p.boundingCube()
print(p.boxMin, p.boxMax)
fig, ax = plt.subplots()
ax.plot(p.pos[:,0], p.pos[0:,1], ',')
ax.set_aspect('equal')

We'll use a common expression for the optimal smoothing length for a Plummer sphere:

In [ ]:
maxLeafSize = 32
epsSmoothing = 0.98*p.N**(-0.26)
B = BHtree(p, maxLeafSize, epsSmoothing)

In [ ]:
tic = time.perf_counter()
B.accAll(0.75)
toc = time.perf_counter()
print(f"elapsed time: {toc-tic} seconds")

In [ ]:
rr = np.logspace(-3, 0, 400)

fig, ax = plt.subplots(1,2)
r = np.linalg.norm(p.pos,axis=1)
a = np.linalg.norm(p.acc,axis=1)
ax[0].loglog(r, a, '.')
ax[0].loglog(rr, rr/((rr**2 + epsSmoothing**2)**(1.5)), 'r')

ax[1].loglog(r, -p.pot, '.')
ax[1].plot(rr, 1.0/((rr**2 + epsSmoothing**2)**(0.5)), 'r')

Sadly, this implementation in Python is far too slow for practical work. The 16K particle run which produced these plots took more than a minute!

A C++ implementation (with a Python interface) which can achieve a rate of more than $10^6$ particles/second for a Plummer sphere of $N=10^6$ is on D2L. Examples of its use are in the notebooks PlummerExample and GalaxyCollide.